In [1]:
import os
import sys
import json
from dotenv import load_dotenv
load_dotenv()

# Add the parent directory
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Kentro

### GetPaySlipTool

In [8]:
from tools.GetPayslipTool import get_payslip
candidate_id = os.environ.get("KENTRO_TEST_CANDIDATE_ID")

In [3]:
print(candidate_id)

100102195


In [5]:
import os
import sys

# Add project root to path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional

# Import the client and its Pydantic models
from clients.kentro import KentroClient, Payslip, PayslipFile

# Initialize the client once
client = KentroClient()

class PayslipInput(BaseModel):
    employee_email: str = Field(description="The flex worker's email address.")
    latest: Optional[bool] = Field(default=False, description="Set to True to retrieve only the single most recent payslip file.")
    payslip_id: Optional[int] = Field(default=None, description="A specific PaySlipId to retrieve. If provided, 'latest' is ignored.")
    start_date: Optional[str] = Field(default=None, description="The start date (YYYY-MM-DD) for a date range query.")
    end_date: Optional[str] = Field(default=None, description="The end date (YYYY-MM-DD) for a date range query.")

@tool(args_schema=PayslipInput)
def get_payslip_tool( 
    latest: Optional[bool] = False, 
    payslip_id: Optional[int] = None,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None
) -> Dict[str, Any]:
    """
    Retrieves payslip information for a flex worker from the Kentro (Pivoton) API.
    Can fetch a specific file, the latest file, or a list of payslips.
    """
    try:
            
        # Case 1: A specific payslip_id is requested
        if payslip_id:
            file_obj: Optional[PayslipFile] = client.get_payslip_file(candidate_id, payslip_id)
            if not file_obj:
                return {"error": f"Payslip file with ID {payslip_id} not found."}
            # Return as a list for consistent data structure
            return {"payslips": [file_obj.dict()]}

        # Case 2: The "latest" payslip is requested
        if latest:
            payslip_list: List[Payslip] = client.get_payslips(candidate_id, start_date, end_date)
            if not payslip_list:
                return {"payslips": [], "message": "No payslips were found."}
                
            # Sort to find the latest payslip
            payslip_list.sort(key=lambda x: x.entry_date, reverse=True)
            latest_payslip = payslip_list[0]
            
            # Fetch the actual file for the latest slip
            file_obj: Optional[PayslipFile] = client.get_payslip_file(candidate_id, latest_payslip.id)
            if not file_obj:
                 return {"error": f"Found latest payslip (ID: {latest_payslip.id}) but could not fetch its file."}

            return {"payslips": [file_obj.dict()]}

        # Case 3: A list of payslips is requested
        payslip_list: List[Payslip] = client.get_payslips(candidate_id, start_date, end_date)
        
        # Serialize the Pydantic models back into dictionaries
        return payslip_list

    except Exception as e:
        # Return a structured error dictionary
        return {"error": f"An unexpected error occurred while retrieving payslips: {str(e)}"}

In [10]:
get_payslip(candidate_id)

Error calling Kentro API endpoint /candidates: 422 Client Error: Unprocessable Entity for url: https://api.flexberichten.nl/api/v1/candidates?EmailAddress=100102195
KentroClient: Failed to retrieve CandidateId: 422 Client Error: Unprocessable Entity for url: https://api.flexberichten.nl/api/v1/candidates?EmailAddress=100102195


{'error': 'Candidate not found: 100102195'}

In [7]:
pay_slip = get_payslip_tool(candidate_id)
pay_slip

/var/folders/v8/38r93dkx5lgg5yxq0bq5vc3w0000gn/T/ipykernel_16121/1288513423.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  pay_slip = get_payslip_tool(candidate_id)
Error parsing payslip file: 1 validation error for PayslipFile
Name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


{'error': 'Found latest payslip (ID: 1209941) but could not fetch its file.'}

### GetReservationsTool

In [3]:
from tools.GetReservationsTool import get_reservations

### GetContractsTool

In [4]:
from tools.GetContractsTool import get_contracts

# Planbition

### GetScheduleTool

In [ ]:
from tools.GetScheduleTool import get_schedule

In [ ]:
TEST_EMPLOYEE_NUMBER = "RE0036443" 
TEST_START_DATE = "2025-05-01"
TEST_END_DATE = "2025-05-31"

schedule = get_schedule.invoke({
            "employee_number": TEST_EMPLOYEE_NUMBER,
            "start_date": TEST_START_DATE,
            "end_date": TEST_END_DATE
        })

print(f"Success! Found {len(schedule)} schedule items.")

Success! Found 7 schedule items.


In [2]:
for item in schedule[0]:
    print(item)

('id', 'DEF894FC-1A82-493D-AE67-7FD0C2E94177_627B699E-0FA6-4153-AC90-C32EBFC230DD_280520250730')
('employeeNumber', 'RE0036443')
('personId', 'RE0036443')
('organisationStructureId', 'Bleckmann Nederland - Nijverheidsstraat')
('organisationStructureName', 'Bleckmann Nederland - Nijverheidsstraat')
('customerId', '0FDD57AF-5DA2-432A-B95F-C158CEBC822A')
('customerName', 'Bleckmann Nederland - Nijverheidsstraat')
('customerStructureName', 'Ruud')
('shiftId', 'F40D2C95-433D-4446-A200-E3A6D5E1A215')
('shiftName', 'Ruud Reco')
('shiftQualificationDemandId', '627B699E-0FA6-4153-AC90-C32EBFC230DD')
('shiftQualificationDemandName', 'Ruud Reco')
('scheduleEmployeeAdministrativeComponents', [])
('startTime', {'year': 2025, 'month': 5, 'day': 28, 'hour': 7, 'minute': 30, 'second': 0, 'millisecond': 0})
('endTime', {'year': 2025, 'month': 5, 'day': 28, 'hour': 16, 'minute': 30, 'second': 0, 'millisecond': 0})
('plannerRemarkPublic', None)
('plannerRemarkPrivate', None)
('employeeRemark', None)
('is

In [3]:
schedule[0].customerName

'Bleckmann Nederland - Nijverheidsstraat'